In [89]:
import plotly.graph_objects as go
import shinybroker as sb
import numpy as np

asset = sb.Contract({
    'symbol': "MSTR",
    'secType': "STK",
    'exchange': "SMART",
    'currency': "USD"
})

benchmark = sb.Contract({
    'symbol': "VOO",
    'secType': "STK",
    'exchange': "SMART",
    'currency': "USD"
})

def fetch_security_historical_data(security_info):
    historical_data = sb.fetch_historical_data(
        security_info,
        barSizeSetting="1 day",
        durationStr="1 Y"
    )
    return historical_data["hst_dta"]["close"]

def calculate_geometric_returns(close_prices):
    close_prices = np.array(close_prices)
    geometric_returns = np.log(close_prices[1:] / close_prices[:-1])
    return geometric_returns

def calculate_arithmetic_returns(close_prices):
    close_prices = np.array(close_prices)
    arithmetic_returns = np.diff(close_prices) / close_prices[:-1]
    return arithmetic_returns

def calculate_beta(asset_return, benchmark_return):
    cov = np.cov(asset_return, benchmark_return)
    cov_xy = cov[0, 1]
    var = np.var(benchmark_return, ddof=1)
    beta = cov_xy / var
    return beta

def calculate_alpha(asset_return, benchmark_return):
    mean_asset_return = np.mean(asset_return)
    mean_benchmark_return = np.mean(benchmark_return)
    beta = calculate_beta(asset_return, benchmark_return)
    alpha = mean_asset_return - beta * mean_benchmark_return
    return alpha

def calculate_best_fit_range(data1, data2, padding_factor=0.1):
    combined_data = np.concatenate((data1, data2))
    data_min = np.min(combined_data)
    data_max = np.max(combined_data)
    range_padding = (data_max - data_min) * padding_factor
    best_min = data_min - range_padding
    best_max = data_max + range_padding
    range_span = best_max - best_min
    tick_increment = round(range_span / 10, 4)  # Divide range into ~10 ticks
    return best_min, best_max, tick_increment

def calculate_geometric_mean(returns):
    compounding_factor = np.exp(np.mean(returns))
    return compounding_factor - 1

def calculate_arithmetic_mean(returns):
    return np.mean(returns)

def calculate_volatility(returns):
    return np.std(returns, ddof=1)
    
    
# Fetch data for both securities
asset_close_data = fetch_security_historical_data(asset)
benchmark_close_data = fetch_security_historical_data(benchmark)

# Calculate the geometric and arithemetic returns 
asset_geometric_returns = calculate_geometric_returns(asset_close_data)
benchmark_geometric_returns = calculate_geometric_returns(benchmark_close_data)
asset_arithmetic_returns = calculate_arithmetic_returns(asset_close_data)
benchmark_arithmetic_returns = calculate_arithmetic_returns(benchmark_close_data)

# Calculate alpha/beta for geometric and arithmetic returns
geo_beta = calculate_beta(asset_geometric_returns, benchmark_geometric_returns)
geo_alpha = calculate_alpha(asset_geometric_returns, benchmark_geometric_returns)
print(f"Geometric - alpha: {geo_alpha} | beta: {geo_beta}")



Geometric - alpha: 0.004922568080105377 | beta: 3.334151383984013


In [90]:
def create_alpha_beta_plot(asset_returns, benchmark_returns, alpha, beta, title, color, line_color, asset_geo_mean, benchmark_geo_mean, asset_arith_mean, benchmark_arith_mean, asset_volatility, benchmark_volatility):
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=benchmark_returns,
        y=asset_returns,
        mode='markers',
        name='Returns',
        marker=dict(size=7, color=color, opacity=0.7)
    ))

    trendline = beta * benchmark_returns + alpha
    fig.add_trace(go.Scatter(
        x=benchmark_returns,
        y=trendline,
        mode='lines',
        name=f'Trendline (Beta={beta:.5f}, Alpha={alpha:.5f})',
        line=dict(color=line_color, dash='solid')
    ))

    metrics_text = (
        f"Geometric Mean:\n"
        f"  Asset: {asset_geo_mean:.5f}\n"
        f"  Benchmark: {benchmark_geo_mean:.5f}<br>"
        f"Arithmetic Mean:\n"
        f"  Asset: {asset_arith_mean:.5f}\n"
        f"  Benchmark: {benchmark_arith_mean:.5f}<br>"
        f"Volatility:\n"
        f"  Asset: {asset_volatility:.5f}\n"
        f"  Benchmark: {benchmark_volatility:.5f}<br>"
    )
    
    fig.add_annotation(
        x=0.02,  
        y=0.02,
        text=metrics_text,
        showarrow=False,
        align='left',
        font=dict(size=10),
        bordercolor='black',
        borderwidth=1,
        borderpad=4,
        bgcolor='white',
        opacity=0.8,
    )

    layout = {
        "title": title,
        "xaxis": {
            "title": "Benchmark Returns",
        },
        "yaxis": {
            "title": "Asset Returns",
        },
        "legend_title": "Legend",
        "template": "plotly_white"
    }
    fig.update_layout(**layout)

    return fig

asset_geo_mean = calculate_geometric_mean(asset_geometric_returns)
benchmark_geo_mean = calculate_geometric_mean(benchmark_geometric_returns)
asset_arith_mean = calculate_arithmetic_mean(asset_arithmetic_returns)
benchmark_arith_mean = calculate_arithmetic_mean(benchmark_arithmetic_returns)
asset_volatility = calculate_volatility(asset_geometric_returns)
benchmark_volatility = calculate_volatility(benchmark_geometric_returns)

geo_fig = create_alpha_beta_plot(
    asset_returns=asset_geometric_returns,
    benchmark_returns=benchmark_geometric_returns,
    alpha=geo_alpha,
    beta=geo_beta,
    title="Geometric Returns Alpha and Beta",
    color="blue",
    line_color="red",
    asset_geo_mean=asset_geo_mean,
    benchmark_geo_mean=benchmark_geo_mean,
    asset_arith_mean=asset_arith_mean,
    benchmark_arith_mean=benchmark_arith_mean,
    asset_volatility=asset_volatility,
    benchmark_volatility=benchmark_volatility
)

geo_fig.show()